In [4]:
import string
import os
import numpy as np
import yaml
import re

import tensorflow as tf
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import load_model
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding

In [5]:
num_threads = 5
os.environ["OMP_NUM_THREADS"] = "5"
os.environ["TF_NUM_INTRAOP_THREADS"] = "5"
os.environ["TF_NUM_INTEROP_THREADS"] = "5"
tf.config.threading.set_inter_op_parallelism_threads(num_threads)
tf.config.threading.set_intra_op_parallelism_threads(num_threads)
tf.config.set_soft_device_placement(True)

In [61]:
# with open('/home/tina/Downloads/ai.yml', 'r') as file:
#     data = yaml.safe_load(file)   
# all_texts = []
# for inner_list in data['conversations']:
#     for text in inner_list:
#         all_texts.append(text)
# print(all_texts)        

In [62]:
FILE_NAME = '/home/tina/Downloads/plato.txt'
text = open(FILE_NAME, "r").read()
all_texts = [l for l in text.split('\n') if l != '']

In [64]:
cleaned_texts = []

pattern = r'[^a-zA-Z\s]'
for text in all_texts:
    cleaned_text = re.sub(pattern, '', text)
    cleaned_text = cleaned_text.lower()
    cleaned_texts.append(cleaned_text)
# print(cleaned_texts)

In [65]:
def clean_doc(doc):
    tokens = doc.split()
    tokens = [' ' if w in string.punctuation else w for w in tokens]
    tokens = [word for word in tokens if word.isalpha()]
    tokens = [word.lower() for word in tokens]
    return tokens

In [87]:
LINE_COUNT = 5000
all_tokens = []
i = 0

for text in all_texts:
    if i < LINE_COUNT:
        tokens = clean_doc(text)
        all_tokens.extend(tokens)
        i += 1
    else:
        break

# print(all_tokens)    

In [88]:
number_of_unique_tokens = len(set(all_tokens))

print('Total Tokens: %d' % len(all_tokens))
print('Unique Tokens: %d' % number_of_unique_tokens)
print('These are the first 50 tokens: %s' % all_tokens[:50])

Total Tokens: 52556
Unique Tokens: 5135
These are the first 50 tokens: ['the', 'project', 'gutenberg', 'ebook', 'of', 'the', 'by', 'plato', 'this', 'ebook', 'is', 'for', 'the', 'use', 'of', 'anyone', 'anywhere', 'at', 'no', 'cost', 'and', 'with', 'almost', 'no', 'restrictions', 'you', 'may', 'copy', 'give', 'it', 'away', 'or', 'it', 'under', 'the', 'terms', 'of', 'the', 'project', 'gutenberg', 'license', 'included', 'with', 'this', 'ebook', 'or', 'online', 'at', 'the', 'republic']


In [89]:
SEQUENCE_LENGTH = 4
LENGTH = SEQUENCE_LENGTH + 1

sequences = list()
for i in range(LENGTH, len(all_tokens)):
    seq = all_tokens[i-LENGTH:i]
    line = ' '.join(seq)
    sequences.append(line)

print ('Total Sequences: %d' % len(sequences))
print ('This is the first sequence: {0}'.format(sequences[0]))

Total Sequences: 52551
This is the first sequence: the project gutenberg ebook of


In [90]:
#sequences[:5]

In [91]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(sequences)
sequences = tokenizer.texts_to_sequences(sequences)
vocab_size = number_of_unique_tokens + 1

In [92]:
sequences0 = np.array(sequences)
X, y = sequences0[:,:-1], sequences0[:,-1]
y = to_categorical(y, num_classes=vocab_size)

In [93]:
dimensions_to_represent_word = 100
 
model = Sequential()
model.add(Embedding(vocab_size, SEQUENCE_LENGTH, input_length=SEQUENCE_LENGTH))
model.add(LSTM(128, return_sequences=False))
# model.add(LSTM(100))
model.add(Dense(128, activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 4, 4)              20544     
                                                                 
 lstm_4 (LSTM)               (None, 128)               68096     
                                                                 
 dense_4 (Dense)             (None, 128)               16512     
                                                                 
 dense_5 (Dense)             (None, 5136)              662544    
                                                                 
Total params: 767696 (2.93 MB)
Trainable params: 767696 (2.93 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [94]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, batch_size=512, epochs=300)
model.save("./data/chat-model-prediction.mdl")

2023-09-18 17:20:37.413325: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 1079607744 exceeds 10% of free system memory.


Epoch 1/300
103/103 [==============================] - 6s 44ms/step - loss: 6.8281 - accuracy: 0.0941
Epoch 2/300
103/103 [==============================] - 4s 43ms/step - loss: 5.9307 - accuracy: 0.0951
Epoch 3/300
103/103 [==============================] - 4s 43ms/step - loss: 5.9131 - accuracy: 0.0951
Epoch 4/300
103/103 [==============================] - 4s 43ms/step - loss: 5.9025 - accuracy: 0.0951
Epoch 5/300
103/103 [==============================] - 4s 43ms/step - loss: 5.8934 - accuracy: 0.0951
Epoch 6/300
103/103 [==============================] - 4s 43ms/step - loss: 5.8846 - accuracy: 0.0951
Epoch 7/300
103/103 [==============================] - 4s 43ms/step - loss: 5.8734 - accuracy: 0.0951
Epoch 8/300
103/103 [==============================] - 4s 43ms/step - loss: 5.8578 - accuracy: 0.0951
Epoch 9/300
103/103 [==============================] - 4s 43ms/step - loss: 5.8310 - accuracy: 0.0951
Epoch 10/300
103/103 [==============================] - 4s 43ms/step - loss: 5.798

INFO:tensorflow:Assets written to: ./data/chat-model-prediction.mdl/assets


In [95]:
print (X.shape)
prediction = model.predict(X[0].reshape(1,SEQUENCE_LENGTH))
print (prediction.shape)
# print (prediction)

(52551, 4)
1/1 [==============================] - 0s 259ms/step
(1, 5136)


In [96]:
loaded_model = load_model("./data/chat-model-prediction.mdl")
loaded_model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 4, 4)              20544     
                                                                 
 lstm_4 (LSTM)               (None, 128)               68096     
                                                                 
 dense_4 (Dense)             (None, 128)               16512     
                                                                 
 dense_5 (Dense)             (None, 5136)              662544    
                                                                 
Total params: 767696 (2.93 MB)
Trainable params: 767696 (2.93 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [104]:
test_texts = [
    'the extent to which',
    'many affinities may be',
    'empty vessel makes the',
    "republic is the vehicle",
    'who suited to raise the',
    'suited to raise the question',
    'to raise the question and',
    'raise the question and preparing'
]

test_sequences = [clean_doc(text) for text in test_texts]
test_sequences = tokenizer.texts_to_sequences(test_sequences)

for i, test_sequence in enumerate(test_sequences):
    if len(test_sequence) < SEQUENCE_LENGTH:
        continue
    input_sequence = test_sequence[-SEQUENCE_LENGTH:]
    input_sequence = np.array(input_sequence).reshape(1, -1)  
    predicted_probs = loaded_model.predict(input_sequence)
    predicted_word_index = np.argmax(predicted_probs)
    print(test_texts[i], test_sequence, input_sequence, predicted_word_index)
    
    predicted_word = tokenizer.index_word[predicted_word_index]
    print(test_texts[i], ' => ', predicted_word)
    print('\n')


1/1 [==============================] - 0s 14ms/step
who suited to raise the [29, 1144, 4, 1145, 1] [[1144    4 1145    1]] 158
who suited to raise the  =>  question


1/1 [==============================] - 0s 17ms/step
suited to raise the question [1144, 4, 1145, 1, 158] [[   4 1145    1  158]] 3
suited to raise the question  =>  and


1/1 [==============================] - 0s 15ms/step
to raise the question and [4, 1145, 1, 158, 3] [[1145    1  158    3]] 1172
to raise the question and  =>  preparing


1/1 [==============================] - 0s 15ms/step
raise the question and preparing [1145, 1, 158, 3, 1172] [[   1  158    3 1172]] 4
raise the question and preparing  =>  to


